Бібліотеки, математичні та допоміжні функції

In [2]:
import random

def print_val(name, val):
    if isinstance(val, int):
        print(f"{name} (Dec): {val}")
        print(f"{name} (Hex): {hex(val)[2:].upper()}")
    else:
        print(f"{name}: {val}")
    print("-" * 20)

#Розширений алгоритм Евкліда (знаходження НСД та коефіцієнтів)
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    else:
        g, y, x = extended_gcd(b % a, a)
        return g, x - (b // a) * y, y

#Знаходження оберненого елемента за модулем
def mod_inverse(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise Exception('Обернений елемент не існує (числа не взаємно прості)')
    else:
        return x % m

#Швидке піднесення до степеня за модулем (схема Горнера)
def power_mod(base, exponent, modulus):
    result = 1
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent // 2
        base = (base * base) % modulus
    return result

Тести на простоту та генерація простих чисел

In [3]:
#Тест Міллера-Рабіна на простоту
def miller_rabin_test(n, k=40):
    if n == 2 or n == 3: return True
    if n % 2 == 0 or n < 2: return False

    #Представляємо n-1 як d * 2^s
    s = 0
    d = n - 1
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = power_mod(a, d, n)
        if x == 1 or x == n - 1:
            continue
        
        for _ in range(s - 1):
            x = power_mod(x, 2, n)
            if x == n - 1:
                break
        else:
            return False #Число складене
    return True #Число просте

#Генерація простого числа заданої бітової довжини
def generate_prime(bits):
    while True:
        #Генеруємо непарне число
        candidate = random.getrandbits(bits)
        candidate |= (1 << bits - 1) | 1 

        #Швидкий тест пробних ділень на малі прості числа 
        if any(candidate % p == 0 for p in [3, 5, 7, 11, 13, 17, 19, 23, 29]):
            continue

        #Основний тест
        if miller_rabin_test(candidate):
            return candidate

Робота з текстом (Text <-> Int)

In [4]:
#Перетворення тексту в число
def TextToInt(text):
    return int.from_bytes(text.encode('utf-8'), byteorder='big')

#Перетворення числа в текст
def IntToText(number):
    #Визначаємо кількість байтів
    num_bytes = (number.bit_length() + 7) // 8
    if num_bytes == 0: return ""
    try:
        return number.to_bytes(num_bytes, byteorder='big').decode('utf-8')
    except:
        return "[Дані не є текстом utf-8]"

Генерація ключів RSA

In [5]:
def GenerateKeyPair(bits=256):
    #Генеруємо p та q
    p = generate_prime(bits)
    q = generate_prime(bits)
    while p == q:
        q = generate_prime(bits)
    
    #Обчислюємо модуль n та функцію Ойлера phi
    n = p * q
    phi = (p - 1) * (q - 1)
    
    #Обираємо e (зазвичай 65537) 
    e = 65537
    #Якщо раптом e не взаємно просте з phi (малоймовірно для 65537), шукаємо інше
    while extended_gcd(e, phi)[0] != 1:
        e = random.randrange(3, phi - 1, 2)

    #Обчислюємо d
    d = mod_inverse(e, phi)
    
    #Повертаємо ((e, n), (d, p, q))
    return ((e, n), (d, p, q))

Основні процедури (Encrypt, Decrypt, Sign, Verify)

In [6]:
#Шифрування: C = M^e mod n
def Encrypt(message_int, public_key):
    e, n = public_key
    if message_int >= n:
        raise ValueError("Повідомлення завелике для цього ключа")
    return power_mod(message_int, e, n)

#Розшифрування: M = C^d mod n
def Decrypt(ciphertext_int, private_key):
    d, p, q = private_key
    n = p * q
    return power_mod(ciphertext_int, d, n)

#Підпис: S = M^d mod n
def Sign(message_int, private_key):
    d, p, q = private_key
    n = p * q
    return power_mod(message_int, d, n)

#Перевірка підпису: M == S^e mod n
def Verify(message_int, signature, public_key):
    e, n = public_key
    check = power_mod(signature, e, n)
    return check == message_int

Протокол обміну ключами

In [7]:
#А формує пакет для В: (E_B(k), E_B(Sign_A(k)))
def SendKey(k, private_key_A, public_key_B):
    e_B, n_B = public_key_B
    d_A, p_A, q_A = private_key_A
    n_A = p_A * q_A
    
    #Перевірка умови n1 >= n [cite: 135]
    if n_B < n_A:
        raise ValueError("Помилка: Модуль отримувача менший за модуль відправника.")

    #А підписує k
    S = Sign(k, private_key_A)
    
    #А шифрує k та підпис S ключем В
    k1 = Encrypt(k, public_key_B)
    S1 = Encrypt(S, public_key_B)
    
    return k1, S1

#В отримує пакет, розшифровує і перевіряє підпис
def ReceiveKey(k1, S1, private_key_B, public_key_A):
    #В розшифровує своїм ключем
    k_received = Decrypt(k1, private_key_B)
    S_received = Decrypt(S1, private_key_B)
    
    #В перевіряє підпис ключем А
    is_valid = Verify(k_received, S_received, public_key_A)
    
    return k_received, is_valid

Головний тестік

In [9]:
def main_lab_scenario():
    print("ЛАБОРАТОРНА РОБОТА RSA\n")
    
    print(">>> 1. Генерація ключів...")
    
    #Ключі Абонента А
    pub_A, priv_A = GenerateKeyPair(256)
    n_A = pub_A[1]
    
    #Ключі Абонента В (з умовою n_B >= n_A)
    while True:
        pub_B, priv_B = GenerateKeyPair(256)
        n_B = pub_B[1]
        if n_B >= n_A:
            break
    
    print("[Абонент А]")
    print_val("Public Key (e)", pub_A[0])
    print_val("Modulus (n)", pub_A[1])
    print_val("Private Key (d)", priv_A[0])
    
    print("[Абонент В]")
    print_val("Public Key (e1)", pub_B[0])
    print_val("Modulus (n1)", pub_B[1])
    print_val("Private Key (d1)", priv_B[0])

    print("\n>>> 2. Шифрування тексту (A -> B)")
    text_message = "Hello IPT"
    print(f"Вхідний текст: {text_message}")
    
    #Перетворення в число
    m_int = TextToInt(text_message)
    print_val("Текст як число", m_int)
    
    #Шифрування (для В)
    c_int = Encrypt(m_int, pub_B)
    print_val("Зашифроване повідомлення (Ciphertext)", c_int)
    
    #Розшифрування (В розшифровує)
    decrypted_int = Decrypt(c_int, priv_B)
    print_val("Розшифроване число", decrypted_int)
    
    #Перетворення назад у текст
    decrypted_text = IntToText(decrypted_int)
    print(f"Результат: '{decrypted_text}'")
    assert text_message == decrypted_text
    
    print("\n>>> 3. Цифровий підпис повідомлення")
    #А підписує повідомлення
    signature = Sign(m_int, priv_A)
    print_val("Цифровий підпис S", signature)
    
    #Хтось перевіряє підпис ключем А
    valid = Verify(m_int, signature, pub_A)
    print(f"Перевірка підпису: {'УСПІХ' if valid else 'ПОМИЛКА'}")

    print("\n>>> 4. Протокол обміну сеансовим ключем k")
    #Випадковий ключ k
    k = random.randrange(1, n_A - 1)
    print_val("Згенерований ключ k", k)
    
    #A відправляє B
    k1, S1 = SendKey(k, priv_A, pub_B)
    print("А відправив пакет (k1, S1)")
    print_val("k1 (Encypted k)", k1)
    print_val("S1 (Encrypted Signature)", S1)
    
    #B отримує
    k_received, auth_valid = ReceiveKey(k1, S1, priv_B, pub_A)
    print("В отримав та обробив пакет")
    print_val("Отримане k", k_received)
    print(f"Автентичність відправника (А): {'ПІДТВЕРДЖЕНО' if auth_valid else 'ВІДХИЛЕНО'}")
    
    if k == k_received and auth_valid:
        print("ЛАБОРАТОРНУ ВИКОНАНО")

if __name__ == "__main__":
    main_lab_scenario()

ЛАБОРАТОРНА РОБОТА RSA

>>> 1. Генерація ключів...
[Абонент А]
Public Key (e) (Dec): 65537
Public Key (e) (Hex): 10001
--------------------
Modulus (n) (Dec): 7797209521010394527509602226917169357838896547104690495970780907151493221578392285224416476739982876480449997841036463216425803358694741345522987932893961
Modulus (n) (Hex): 94DFF75B75605AF7C04F9C4E7878058D36A8917900A72BCE35CD8E565632FABADF20AE048C55EAF301D7D936E1CBDB27D2539451FAADCEFE221F115FD5ACC309
--------------------
Private Key (d) (Dec): 569053407067652120559049505643298000191394817962398868459469384910899065853008049270507842966090774190021283540662923703496076934443363161440370388598189
Private Key (d) (Hex): ADD79A10C24F71EA3C36BA5F276D7F8E167560E9DB495A6DC185F6DFFA682D3E9887A7C0340074BBE390F57E291E3B4E3C7C819E1E11CC65C69046039177DAD
--------------------
[Абонент В]
Public Key (e1) (Dec): 65537
Public Key (e1) (Hex): 10001
--------------------
Modulus (n1) (Dec): 869501337396403340109915216803962620872599779117830208928

Тестік для сайту

In [10]:
def integration_test_with_site():
    print("ТЕСТіК СУМІСНОСТІ З ВЕБ-САЙТОМ===\n")

    #Встав сюди ключі з попереднього блоку (Modulus та Private D) для дешифрування та підпису
    my_modulus_hex = "EF162B3F6A3A06750C5B79269345AE75425484D73FBB7FDD9D90EC05352B2F652612C5FD3E06E91C1ACE89C06B454F1408FB6D85773251C3C2A88FFF66427723"
    my_private_d_hex = "9BB669864CADEBE4B19EA5C1DCEAE61B928EAF51ED3D35A2E56B2C7C9BCB3B7AB7ACA2F762110AAF3A3FB35A1CD20C8FF6783A6201DBECCA0FF7D8399702A911"

    #Встав сюди ШИФРОТЕКСТ з сайту(отриманий ключем попереднього блоку), який треба розшифрувати
    ciphertext_hex_input = "9D7169D0D4C3738D04E16A90CA009F2D289A617B252BD3DED9915196383F300E5C106817231F3C95C0C78A190C29E2F4B682AAB5A50302BDB6DDDEBE70E104AE"

    #Сюди дані згенеровані ключем з сайту
    site_modulus_hex = "C212EBB1BFFA7487D068F59EA6AE15B82D8058598FF4B99375495AB16F65AA6B"
    site_exponent_hex = "10001" 
    site_signature_hex = "68B253CF9A1AF839211F3318F39CF59006C631451B93E3C4E01A647E01B49A78"
    
    test_message = "Hello IPT"

    try:
        #Ключі сайту
        n_site = int(site_modulus_hex.strip(), 16)
        e_site = int(site_exponent_hex.strip(), 16)
        s_site = int(site_signature_hex.strip(), 16)
        site_public_key = (e_site, n_site)
        
        print(f"Ключ сайту зчитано.")

        #Наші ключі
        if my_modulus_hex and my_private_d_hex:
            try:
                n_my = int(my_modulus_hex.strip(), 16)
                d_my = int(my_private_d_hex.strip(), 16)
                my_priv_key_struct = (d_my, n_my, 1) #(d, n, 1) для сумісності
                print("Ваші ключі зчитано.")
            except ValueError:
                print("ПОМИЛКА: Невірний формат ваших ключів (перевірте, чи це Hex).")
                my_priv_key_struct = None
        else:
            my_priv_key_struct = None
            print("УВАГА: Ваші ключі порожні.")
            
    except ValueError:
        print("ПОМИЛКА: Невірний формат ключів сайту.")
        return


    print(f"\n>>> Тест 1: Шифрування '{test_message}' (для Сайту)")
    m_int = TextToInt(test_message)
    c_local = Encrypt(m_int, site_public_key)
    print(hex(c_local)[2:].upper())

    print(f"\n>>> Тест 2: Перевірка підпису сайту")
    is_valid = Verify(m_int, s_site, site_public_key)
    print(f"Результат: {'УСПІХ' if is_valid else 'ПОМИЛКА'}")


    print(f"\n>>> Тест 3: Дешифрування шифротексту з сайту")
    
    if my_priv_key_struct:
        try:
            c_input = int(ciphertext_hex_input.strip(), 16)
            m_decrypted = Decrypt(c_input, my_priv_key_struct)
            text_result = IntToText(m_decrypted)
            
            print("Результат:")
            print(f"HEX: {hex(m_decrypted)[2:].upper()}")
            print(f"TEXT: '{text_result}'")
        except ValueError:
            print("ПОМИЛКА: Некоректний Hex-рядок у змінній ciphertext_hex_input.")
        except Exception as e:
            print(f"Помилка при дешифруванні: {e}")
    else:
        print("ПРОПУЩЕНО: Не задані ваші ключі.")

    print(f"\n>>> Тест 4: Генерація вашого підпису")
    if my_priv_key_struct:
        my_sig = Sign(m_int, my_priv_key_struct)
        print(hex(my_sig)[2:].upper())
    else:
        print("ПРОПУЩЕНО: Не задані ваші ключі.")

if __name__ == "__main__":
    integration_test_with_site()

ТЕСТіК СУМІСНОСТІ З ВЕБ-САЙТОМ===

Ключ сайту зчитано.
Ваші ключі зчитано.

>>> Тест 1: Шифрування 'Hello IPT' (для Сайту)
860E6A65BDCDF89BF7BCC718B1466E31F74A37EA9EB39B378D63DDD139A63B5C

>>> Тест 2: Перевірка підпису сайту
Результат: УСПІХ

>>> Тест 3: Дешифрування шифротексту з сайту
Результат:
HEX: 48656C6C6F20495054
TEXT: 'Hello IPT'

>>> Тест 4: Генерація вашого підпису
C10E5F77B1C051E6122767E8FE29910DA962273A8C7F912D01C2EDFE036E8E2A78CA41266BCECABA3D6BA8E44E2476D006FD4EE081F0A632B2C9CF8ABD34D022
